# Game and Table Initialization

In [1]:
'''poker test'''

# pylint: disable=E1101, E1601, W0612

import db_funcs
import db_moves

import pdeck

In [2]:
# GAME PARAMETERS

number_of_players = 6
starting_stack = 1000
small_blind = 10
big_blind = 2 * small_blind

db_funcs.sql_delete_all(table='games')
db_funcs.sql_delete_all(table='history')
db_funcs.sql_delete_all(table='decision_points')
db_funcs.sql_delete_all(table='possible_moves')
db_funcs.sql_insert_games(values_list=[number_of_players, starting_stack, small_blind, big_blind])

In [3]:
# TABLE INITIALIZATION

TABLE = db_funcs.table_init(nr_of_players=number_of_players, starting_stack=starting_stack)
POT = TABLE[0]
PLAYERS = TABLE[1]

In [4]:
# GAME INITIALIZATION

# # shift the players position by 1
for player in PLAYERS:

    player.add_position((player.position_nr() + 1) % len(PLAYERS))

# # create deck and board
DECK = pdeck.Deck()
BOARD = DECK.make_board()

# # create hands and add them to players
HANDS = [DECK.make_hand() for player in PLAYERS]
for i in range(len(PLAYERS)):

    PLAYERS[i].add_hand(HANDS[i])

# # define players order
PLAYER_ORDER = [0 for player in PLAYERS]
for player in PLAYERS:

    PLAYER_ORDER[player.position_nr()] = player

# # define players move flags
PLAYER_MOVE_FLAGS = [1 for player in PLAYERS]

In [5]:
# PREFLOP PHASE

# # small blind and big blind moves
db_moves.small_blind_move(player=PLAYER_ORDER[0], pot=POT, amount=small_blind)
db_moves.big_blind_move(player=PLAYER_ORDER[1], pot=POT, amount=big_blind)

In [6]:
# # further moves
phase = 0
nr = 2
for i in range(2, 6):
    
    move = db_funcs.decision_point(hand=PLAYER_ORDER[i].show_player_hand_db(), stack=PLAYER_ORDER[i].stack(), \
        pot=POT.show_pot(), position=PLAYER_ORDER[i].position_nr(), phase=phase, nr=nr)
    if move == 'fold':
        db_moves.fold_move(player=PLAYER_ORDER[i], pot=POT, phase=phase, nr=nr)
    elif move == 'check':
        db_moves.check_move(player=PLAYER_ORDER[i], pot=POT, phase=phase, nr=nr)
    elif move == 'call':
        call_amount = 60 # calculation for call_amount
        db_moves.call_move(player=PLAYER_ORDER[i], pot=POT, phase=phase, nr=nr, amount=call_amount)
    elif move == 'bet':
        bet_amount = 60 # calculation for bet_amount
        db_moves.bet_move(player=PLAYER_ORDER[i], pot=POT, phase=phase, nr=nr, amount=bet_amount)
    elif move == 'raise':
        raise_amount = 60 # calculation for bet_amount
        db_moves.raise_move(player=PLAYER_ORDER[i], pot=POT, phase=phase, nr=nr, amount=raise_amount)
    elif move == 'allin':
        db_moves.allin_move(player=PLAYER_ORDER[i], pot=POT, phase=phase, nr=nr, amount=PLAYER_ORDER[i].stack())
    else:
        pass
    
    # save move information for every players

    nr = nr + 1